In [241]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
from plotly.graph_objs import *
import pandas as pd
import numpy as np

In [242]:
df = pd.read_csv ("zizhendahaoren.csv",
                  index_col = 0)

In [243]:
df.head()


,State,specify,Violent_crime,Murder_and_nonnegligent_manslaughter,Rape,Robbery,Aggravated_assault,Property_crime,Burglary,Larceny-theft,Motor_vehicle_theft,year
10,ALABAMA,"Rate per 100,000 inhabitants",532.3,8.4,39.4,96.4,388.2,2947.8,700.5,2006.3,241.1,2016
19,ALASKA,"Rate per 100,000 inhabitants",804.2,7.0,141.9,114.6,540.6,3353.0,546.3,2394.7,412.1,2016
28,ARIZONA,"Rate per 100,000 inhabitants",470.1,5.5,47.5,101.8,315.4,2978.4,544.4,2168.1,265.8,2016
39,ARKANSAS,"Rate per 100,000 inhabitants",550.9,7.2,71.7,70.9,401.0,3268.6,795.5,2233.6,239.4,2016
48,CALIFORNIA,"Rate per 100,000 inhabitants",445.3,4.9,34.9,139.6,265.9,2553.0,479.8,1623.0,450.3,2016


In [244]:
crime_cols = ['Violent_crime', 'Murder_and_nonnegligent_manslaughter', 'Rape', 'Robbery', 
              'Aggravated_assault', 'Property_crime', 'Burglary', 'Larceny-theft', 'Motor_vehicle_theft']
for col in crime_cols:
    if df[col].dtype != 'float64':
        df[col] = df[col].astype('float')

In [245]:
df['all_crime'] = df['Violent_crime'] + df['Murder_and_nonnegligent_manslaughter'] + df['Rape'] + df['Robbery'] +\
    df['Aggravated_assault'] + df['Property_crime'] + df['Burglary'] + df['Larceny-theft'] + df['Motor_vehicle_theft']

In [246]:
code_map = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/2011_us_ag_exports.csv')[['code', 'state']]


In [247]:
state_map = {}
for i in range(code_map.shape[0]):
    code = code_map.iloc[i, 0]
    state = code_map.iloc[i, 1].upper()
    state_map[state] = code  

In [248]:
state_map['CALIFORNIA'] = 'CA'
state_map['DISTRICT OF COLUMBIA'] = 'DC'
state_map['PUERTO RICO'] = 'PR'

In [249]:
df['Code'] = df.State.map(state_map)

In [250]:
df = df[(df['State'] != 'DISTRICT OF COLUMBIA') & (df['State'] != 'PUERTO RICO')]

In [251]:
df['text'] = "per 100,000 inhabitants" + '<br>' +  df['State']

In [252]:
df[df['Code'].isna()]

,State,specify,Violent_crime,Murder_and_nonnegligent_manslaughter,Rape,Robbery,Aggravated_assault,Property_crime,Burglary,Larceny-theft,Motor_vehicle_theft,year,all_crime,Code,text


In [253]:
state_map['ALABAMA']

'AL'

In [254]:
df.tail()

,State,specify,Violent_crime,Murder_and_nonnegligent_manslaughter,Rape,Robbery,Aggravated_assault,Property_crime,Burglary,Larceny-theft,Motor_vehicle_theft,year,all_crime,Code,text
460,VIRGINIA,"Rate per 100,000 inhabitants",195.6,4.6,27.9,53.0,110.2,1866.5,254.6,1515.2,96.8,2015,4124.4,VA,"per 100,000 inhabitants<br>VIRGINIA"
470,WASHINGTON,"Rate per 100,000 inhabitants",284.4,2.9,37.7,76.0,167.8,3463.8,711.2,2378.0,374.7,2015,7496.5,WA,"per 100,000 inhabitants<br>WASHINGTON"
481,WEST VIRGINIA,"Rate per 100,000 inhabitants",337.9,3.8,36.4,41.2,256.4,2020.0,497.3,1401.3,121.4,2015,4715.7,WV,"per 100,000 inhabitants<br>WEST VIRGINIA"
491,WISCONSIN,"Rate per 100,000 inhabitants",305.8,4.2,30.8,90.7,180.1,1974.0,338.8,1444.8,190.3,2015,4559.5,WI,"per 100,000 inhabitants<br>WISCONSIN"
501,WYOMING,"Rate per 100,000 inhabitants",222.1,2.7,29.5,10.1,179.8,1902.6,300.6,1500.9,101.0,2015,4249.3,WY,"per 100,000 inhabitants<br>WYOMING"


# Single year, single crime

In [240]:
df = df[df['year']==2016]
crime = 'all_crime'
data = [dict(
        type = 'choropleth',
        locations = df['Code'],
        locationmode = 'USA-states',
        z = df[crime],
        text = df['text'],
        #colorscale = [ "#c9d9d3", "#e2e2e2", "#dfccce", "#ddb7b1", "#cc7878", "#933b41", "#550b1d"],
        colorscale = [[0.0, "#550b1d"] ,[0.2, "#933b41"],[0.4, "#cc7878"],\
            [0.6, "#ddb7b1"],[0.8, "#dfccce"],[1.0, "#e2e2e2"]],
        autocolorscale = False,
        reversescale = True,
        marker = dict(
            cmax=max(df[crime]),
            cmin=min(df[crime]),
            line = dict (
                color = 'rgb(255,255,255)',
                width = 2
            ) ),
            colorbar = dict(
                autotick = False,
                tickprefix = '',
                title = 'Crime Rate'),
          ) ]

layout = dict(
    title = 'US Crime Rate: ' + crime,
    geo = dict(
        scope='usa',
        showframe = False,
        showcoastlines = False,
        projection = dict(
            type = 'Mercator'
        )
    )
)

fig = dict( data=data, layout=layout )
iplot( fig, validate=False, filename='d3-world-map' )

# Animation

In [ ]:
# df['all_crime'] = df['Violent_crime'] + df['Murder_and_nonnegligent_manslaughter'] + df['Rape'] + df['Robbery'] +\
#     df['Aggravated_assault'] + df['Property_crime'] + df['Burglary'] + df['Larceny-theft'] + df['Motor_vehicle_theft']

In [331]:
years_sorted = sorted(list(df.year.unique()))

In [344]:
crime = 'Motor_vehicle_theft'
crime_name = 'Motor Vehicle Theft'

figure = {
    'data': [],
    'layout': {},
    'frames': [],}

data = []
for year in years_sorted:
    df_this_year = df[df['year'] == year]
    data_this_year = dict(
            type = 'choropleth',
            locations = df_this_year['Code'],
            locationmode = 'USA-states',
            z = df_this_year[crime],
            text = df_this_year['text'],
            colorscale = [[0.0, "#550b1d"] ,[0.2, "#933b41"],[0.4, "#cc7878"],\
                [0.6, "#ddb7b1"],[0.8, "#dfccce"],[1.0, "#e2e2e2"]],
        
            reversescale = True,
            marker = dict(
                line = dict (
                    color = 'rgb(255,255,255)',
                    width = 2
                ) ),
                colorbar = dict(
                    tickprefix = '',
                    title = 'Crime Rate'),
              )
    data.append(data_this_year)

figure['data'] = [data[0]]
figure['layout'] = dict(
    title = 'US Crime Rate : ' + crime_name,
    geo = dict(
        scope = 'usa',
        showframe = False,
        showcoastlines = False,
        projection = dict(
            type = 'Mercator'
        )
    )
)
sliders_dict = {
    'active': 0,
    'yanchor': 'top',
    'xanchor': 'left',
    'currentvalue': {
        'font': {'size': 20},
        'prefix': 'Year:',
        'visible': True,
        'xanchor': 'right'
    },
    'transition': {'duration': 500, 'easing': 'cubic-in-out'},
    'pad': {'b': 10, 't': 50},
    'len': 0.9,
    'x': 0.1,
    'y': 0,
    'steps': []
}

figure['layout']['updatemenus'] = [
    {
        'buttons': [
            {
                'args': [None, {'frame': {'duration': 500, 'redraw': False},
                         'fromcurrent': True, 'transition': {'duration': 300, 'easing': 'quadratic-in-out'}}],
                'label': 'Play',
                'method': 'animate'
            },
            {
                'args': [[None], {'frame': {'duration': 0, 'redraw': False}, 'mode': 'immediate',
                'transition': {'duration': 0}}],
                'label': 'Pause',
                'method': 'animate'
            }
        ],
        'direction': 'left',
        'pad': {'r': 10, 't': 87},
        'showactive': False,
        'type': 'buttons',
        'x': 0.1,
        'xanchor': 'right',
        'y': 0,
        'yanchor': 'top'
    }
]

figure['frames'] = [{'data': [data[0]], 'name': str(2015)},
                   {'data': [data[1]], 'name': str(2016)},
                   {'data': [data[2]], 'name': str(2017)}]
figure['frames'] = []

for i, year in enumerate(years_sorted):
    slider_step = {'args': [
        [year],
        {'frame': {'duration': 500, 'redraw': False},
         'mode': 'immediate',
         'transition': {'duration': 500}}
         ],
         'label': year,
         'method': 'animate'}
    sliders_dict['steps'].append(slider_step)
    figure['frames'].append({'data': [data[i]], 'name': str(year)})

figure['layout']['sliders'] = [sliders_dict]
plot(figure, show_link=False, filename='us_crime_' + crime)

/Users/SONGZIZHEN/anaconda/envs/py36/lib/python3.6/site-packages/plotly/offline/offline.py:459: UserWarning:

Your filename `us_crime_Motor_vehicle_theft` didn't end with .html. Adding .html to the end of your file.



'file:///Users/SONGZIZHEN/Desktop/DataVisulizaition/Project_New/us_crime_Motor_vehicle_theft.html'